In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/the-office_lines.csv', index_col=0)

In [3]:
df

,Character,Line,Season,Episode_Number
0,Michael,All right Jim. Your quarterlies look very goo...,1,1
1,Jim,"Oh, I told you. I couldn’t close it. So…",1,1
2,Michael,So you’ve come to the master for guidance? Is...,1,1
3,Jim,"Actually, you called me in here, but yeah.",1,1
4,Michael,"All right. Well, let me show you how it’s don...",1,1
...,...,...,...,...
61302,Creed,It all seems so very arbitrary. I applied for...,9,23
61303,Meredith,I just feel lucky that I got a chance to shar...,9,23
61304,Phyllis,I’m happy that this was all filmed so I can r...,9,23
61305,Jim,I sold paper at this company for 12 years. My...,9,23


In [6]:
# We would like to have 1 entry = 1 sentence. not multiple sentences / until the next character speaks
df.iloc[0]['Line']

' All right Jim. Your quarterlies look very good. How are things at the library?'

In [85]:
# Let's split all the multi-sentence lines into single sentences and assign 1 row entry per sentence
splitchars = ['.', '?', '!']
df_sentences = df.copy()
for schar in splitchars:
    df_sentences=df_sentences.assign(Line=df_sentences.Line.str.split(schar)).explode("Line")

In [86]:
# Now let's remove all the empty sentences / spaces that were created from splitting / or already existed in the raw data 
df_sentences.Line = df_sentences.Line.str.replace(r'\s+', ' ', regex=True)
df_sentences = df_sentences[(df_sentences['Line'] != '') & (df_sentences['Line'] != ' ')]

In [87]:
# And re-create a meaningful index
if 'line_id' not in df_sentences.columns:
    df_sentences= df_sentences.reset_index().rename(columns={'index': 'line_id', 'Line': 'Sentence'})

In [89]:
df_sentences['Sentence'].value_counts()

 Yeah                                                 1184
 No                                                   1116
 Okay                                                  953
 Yes                                                   706
 What                                                  610
                                                      ... 
 I like knowing that there’s going to be a break         1
 Yeah, I work hard all day                               1
 It’s a birthday, so what if there’s a lot of them       1
 What is Jim thinking                                    1
 Isn’t that kind of the point                            1
Name: Sentence, Length: 85528, dtype: int64

In [113]:
# We also need to remove 'descriptions of situations' from the sentences e.g., [Michael sings] blabla -> blabla
df_sentences['Sentence'] = df_sentences['Sentence'].str.replace(r"\[.*\]","")
df_sentences = df_sentences[(df_sentences['Sentence'] != ']') 
                            & (df_sentences['Sentence'] != '] ')
                            & (df_sentences['Sentence'] != '”] ')
                           ]

/var/folders/gl/lf_ddy9561j7lbb3hpj3ks2w0000gn/T/ipykernel_8664/3518932128.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_sentences['Sentence'] = df_sentences['Sentence'].str.replace(r"\[.*\]","")


In [ ]:
# Some situation descriptions have a missing "[" and only a closing bracket "]"
# A quick investagtion shows that these are only ~200 sentences and they always START with the situation description
# Thus we can just remove everything BEFORE and incl. the closing bracket
df_sentences['Sentence'] = df_sentences['Sentence'].str.replace(r"\[.*\]","")

In [117]:
df_sentences[df_sentences.Sentence.str.contains(']')]['Sentence'].value_counts()[:20]

] Oh                                                                                                                               2
] Why                                                                                                                              2
] Bummer                                                                                                                           2
] Reese’s Pieces                                                                                                                   2
 Brown] Oh, hey                                                                                                                    1
 As the cab leaves, Pam drives in] She’s my family                                                                                 1
 Pauli Girl Sign, hanging in Ryan’s office]                                                                                        1
 Nervously, Andy laughs with him]                                    

In [115]:
df_sentences.to_csv('../data/the-office_lines_sentences_preprocessed.csv')